In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [3]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

In [4]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [5]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [6]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [7]:
C=1.0
n_splits=5

In [8]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))


C=1.0 0.840 +- 0.008


In [9]:
scores

[0.8420571311945113,
 0.8438638094292147,
 0.8311780052177403,
 0.8301724275756219,
 0.8522638134981644]

In [10]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

## Save Our Model

In [11]:
import pickle

In [12]:
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [13]:
f_out = open(output_file,'wb')
pickle.dump((dv,model),f_out)
f_out.close()

In [14]:
with open(output_file,'wb') as f_out:
    pickle.dump((dv,model),f_out)

In [15]:
import pickle

In [16]:
with open('model_C=1.0.bin','rb') as f_in:
    dv,model=pickle.load(f_in)

In [17]:
dv,model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [18]:
customer={'gender':'female',
'seniorcitizen':0,
'partner':'no',
'dependents':'no',
'tenure':42,
'phoneservice':'yes',
'multiplelines':'no',
'internetservice':'dsl',
'onlinesecurity':'yes',
'onlinebackup':'yes',
'deviceprotection':'yes',
'techsupport':'yes',
'streamingtv':'no',
'streamingmovies':'yes',
'contract':'one_year',
'paperlessbilling':'no',
'paymentmethod':'credit_card_(automatic)',
'monthlycharges':73.9,
'totalcharges':3160.55}

In [19]:
x=dv.transform([customer])

In [20]:
model.predict_proba(x)[0,1]

0.02693707162905431